# Loader

In [1]:
import os

In [34]:
import openai

In [2]:
import sys

In [5]:
from dotenv import load_dotenv

In [35]:
load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

# Folder loading and metadata enrichment

In [10]:
from langchain.document_loaders import DirectoryLoader

In [16]:
loader = DirectoryLoader('FAQ/', glob="*.txt")

In [17]:
documents = loader.load()

In [19]:
for doc in documents:
    print(doc)

page_content='HAND LUGGAGE\n\nAm I always allowed to take my hand luggage with me in the cabin? Your accessory (max. 40 x 30 x 20 cm) is always allowed in the cabin. Place it under the seat in front of you. For larger bags and trolleys (max. 55 x 35 x 25 cm) there is limited space in the cabin. Only the first 70 pieces are therefore allowed on board. The remaining bags and trolleys of this size are taken in at the gate and transported free of charge in the hold. Important for onboard safety! If we transport your hand luggage in the hold, make sure you place your electronics and batteries in your accessory (for example in your handbag or backpack). With electronics, we mean a phone, tablet, e-reader, laptop or e-cigarette, for example. With batteries, we mean loose AA batteries or a power bank, for example. You can read more about this in ‘Allowed in luggage’.\n\nCan I bring food and drinks with me? Yes, you can take food in your hand luggage. However, the same rules apply for drinks as

In [20]:
for doc in documents: 
    doc.metadata["name"] = doc.metadata["source"].split("/")[1].replace("_", " ")[:-4].capitalize()

In [22]:
test_doc.metadata

{'source': 'FAQ/hand_luggage.txt', 'name': 'Hand luggage'}

# Chunking & Vectore Store Creation

In [23]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [24]:
text_splitter = CharacterTextSplitter(chunk_size=600, chunk_overlap=100)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(texts, embeddings)

Created a chunk of size 841, which is longer than the specified 600
Created a chunk of size 749, which is longer than the specified 600
Created a chunk of size 1069, which is longer than the specified 600
Created a chunk of size 1296, which is longer than the specified 600
Created a chunk of size 1020, which is longer than the specified 600
Created a chunk of size 1414, which is longer than the specified 600
Created a chunk of size 1023, which is longer than the specified 600
Created a chunk of size 717, which is longer than the specified 600
Created a chunk of size 893, which is longer than the specified 600
Created a chunk of size 920, which is longer than the specified 600
Created a chunk of size 1473, which is longer than the specified 600
Created a chunk of size 1490, which is longer than the specified 600
Created a chunk of size 1388, which is longer than the specified 600


---

# Retriever Tool

In [25]:
from langchain.agents.agent_toolkits import create_retriever_tool

In [26]:
retriever = db.as_retriever()

In [27]:
tool = create_retriever_tool(
    retriever,
    "search_transavia_FAQ",
    "Searches the Transavia company FAQ and returns documents regarding the company luggage policy"
)
tools = [tool]

In [28]:
tool

Tool(name='search_transavia_FAQ', description='Searches the Transavia company FAQ and returns documents regarding the company luggage policy', args_schema=<class 'langchain.tools.retriever.RetrieverInput'>, func=<bound method BaseRetriever.get_relevant_documents of VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x2b2396950>)>, coroutine=<bound method BaseRetriever.aget_relevant_documents of VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x2b2396950>)>)

---

# Agent Constructor

In [29]:
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent

In [30]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0)

In [31]:
agent_executor = create_conversational_retrieval_agent(
    llm, 
    tools,
    verbose=False
)

---

# Test the Agent

In [32]:
agent_executor({"input": "Hi, I'm Bob"})["output"]

'Hello Bob! How can I assist you today?'

In [33]:
agent_executor({"input": "Please provide a bulletpoint list of the items I'm allowed to bring in my luggage"})["output"]

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-KTalvhb4vzoBFuJF2A18k1TT on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}

In [46]:
agent_executor({"input": "What s my name ?"})



> Entering new AgentExecutor chain...
Your name is Bob.

> Finished chain.


{'input': 'What s my name ?',
 'chat_history': [HumanMessage(content="Hi, I'm Bob"),
  AIMessage(content='Hello Bob! How can I assist you today?'),
  HumanMessage(content="Please provide a bulletpoint list of the items I'm allowed to bring in my luggage"),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "luggage policy"\n}', 'name': 'search_transavia_FAQ'}}),
  FunctionMessage(content='[Document(page_content=\'HAND LUGGAGE\', metadata={\'source\': \'FAQ\\\\hand_luggage.txt\', \'name\': \'Hand luggage\'}), Document(page_content=\'ALLOWED IN LUGGAGE\', metadata={\'source\': \'FAQ\\\\allowed_in_luggage.txt\', \'name\': \'Allowed in luggage\'}), Document(page_content=\'What luggage can I take with me on board? In order to limit contact during boarding and disembarkation, as well as movements in the cabin during the flight, we recommend that you limit your luggage carried on board to the bare minimum. You must stow your own luggage independently. In 

In [47]:
agent_executor({"input": "Can I bring my pet in my luggage?"})



> Entering new AgentExecutor chain...

Invoking: `search_transavia_FAQ` with `{'query': 'bring pet in luggage'}`


[Document(page_content='ALLOWED IN LUGGAGE', metadata={'source': 'FAQ\\allowed_in_luggage.txt', 'name': 'Allowed in luggage'}), Document(page_content='HAND LUGGAGE', metadata={'source': 'FAQ\\hand_luggage.txt', 'name': 'Hand luggage'}), Document(page_content='Can I check in my hand luggage? Please do! You can hand in your hand luggage at the check-in desk at the airport. This leaves your hands free and allows you to go through the security faster. We strictly advise that you take your valuables, such as documents, electronics, batteries and medicines, with you in your accessory (max. 40 x 30 x 20 cm), so that you have them on board. You can retrieve your hand luggage from the baggage carousel after the flight.', metadata={'source': 'FAQ\\hand_luggage.txt', 'name': 'Hand luggage'}), Document(page_content='Can I bring my winter sports equipment with me? If you are only tak

{'input': 'Can I bring my pet in my luggage?',
 'chat_history': [AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "allowed in luggage"\n}', 'name': 'search_transavia_FAQ'}}),
  FunctionMessage(content="[Document(page_content='ALLOWED IN LUGGAGE', metadata={'source': 'FAQ\\\\allowed_in_luggage.txt', 'name': 'Allowed in luggage'}), Document(page_content='HAND LUGGAGE', metadata={'source': 'FAQ\\\\hand_luggage.txt', 'name': 'Hand luggage'}), Document(page_content='Am I always allowed to take my hand luggage with me in the cabin? Your accessory (max. 40 x 30 x 20 cm) is always allowed in the cabin. Place it under the seat in front of you. For larger bags and trolleys (max. 55 x 35 x 25 cm) there is limited space in the cabin. Only the first 70 pieces are therefore allowed on board. The remaining bags and trolleys of this size are taken in at the gate and transported free of charge in the hold. Important for onboard safety! If we transport your hand lug